In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import time
from scipy.fft import fft, ifft

In [ ]:
srate = 128

def time_frequency(data, cmwX, nKern):

    nData   = data.shape[1]
    nConv   = nData + nKern - 1 
    halfwav = (nKern-1)//2

    tf = np.zeros((data.shape[0], cmwX.shape[0], data.shape[1])) #32*10*3200

    for chani in range(data.shape[0]):
        eegX = fft(data[chani, :] , nConv) 
        as_ = ifft(cmwX * eegX[None, :], axis=1) #convo
        as_ = as_[:, halfwav: -halfwav]
        tf[chani, :, :] = np.abs(as_) ** 2 

    return tf


def get_cmwX(nData, freqrange=[1,40], numfrex=42):

    pi = np.pi
    wavtime = np.arange(-2,2-1/srate,1/srate)
    nKern = len(wavtime)
    nConv = nData + nKern - 1
    frex = np.linspace(freqrange[0],freqrange[1],numfrex)
    cmwX = np.zeros((numfrex, nConv), dtype=complex)

    numcyc = np.linspace(3,15,numfrex);
    for fi in range(numfrex):
        s = numcyc[fi] / (2*pi*frex[fi]) # period of a wave = cycle/2*pi*freq
        twoSsquared = (2*s) ** 2^2
        cmw = np.exp(2*1j*pi*frex[fi]*wavtime) * np.exp( (-wavtime**2) / twoSsquared ) #e^2jπft* e^(-t^2/(2s)^2)

        cmwX[fi, :] = fft(cmw, nConv)
        cmwX[fi, :] = cmwX[fi, :] / max(cmwX[fi, :])
        
    return cmwX, nKern, frex

In [ ]:
directory = r'C:\Users\Lilly Pushparani\Downloads\Data\Data\22'


In [ ]:
csv_file_path =r'C:\Users\Lilly Pushparani\Downloads\Data\Data\codes.csv'
codes_df = pd.read_csv(csv_file_path ,header=None)
codes = codes_df[0].tolist()
len(codes)

In [ ]:
index=1
for i in range(1, 361):
    mnist_path = mnist_path = os.path.join(directory,f'{i}.csv')

    mnist_df = pd.read_csv(mnist_path, sep='\t', header=None, nrows=360)
    mnist_df.columns = ['data']
    df = mnist_df
    df["data"] = df["data"].apply(lambda x: [float(i) for i in x.split(",")])

    pnts = df.shape[0]
    
    df["resampled_data"] = df["data"]
    df["data"].shape
    data_array = np.array(df["resampled_data"].tolist())
    data_array = np.reshape(data_array, (-1, 1, data_array.shape[1]))
    data_array = np.mean(data_array, axis=1)
    starting_freq = 1
    end_freq = 10
    num_frequencies = 10
    median_length = 3200
    times = np.linspace(0,2,median_length)
    nData = data_array.shape[1]
    cmwX, nKern, frex = get_cmwX(nData, freqrange=[starting_freq, end_freq], numfrex=num_frequencies)
    tf = time_frequency(data_array, cmwX, nKern)

    fig, axs = plt.subplots(3, 3, figsize=(15, 10))
    for j,ax in enumerate(axs.flat):
      x = random.randint(0, tf.shape[0]-1)
      contour = ax.contourf(times, frex, tf[x,:,:], 40, cmap='jet')
      ax.set_xlabel('Time (s)')
      ax.set_ylabel('Frequencies (Hz)')
      ax.set_title(f"Time Frequency Plot for {'Stress' if codes[x] == 1 else 'No Stress'}")
    fig.tight_layout()
    tf.shape
    path = r'C:\Users\Lilly Pushparani\Downloads\Data\Data\images'
    for k in range(tf.shape[0]):
        print(index)
        code = 0 if codes[index-1] == 0 else 1
        folder_path =  f"{path}/{code}"
        
        os.makedirs(folder_path, exist_ok=True)
        fig, ax = plt.subplots()
        ax.contourf(times, frex, tf[k,:,:], 40, cmap='jet')
        ax.axis('off')
        
        file_path = f"{folder_path}/{index}.png"
        fig.savefig(file_path, bbox_inches='tight', pad_inches=0)
        
        index=index+1
        fig.clf()
        plt.close()

In [ ]:
# CNN